In [75]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from data import *
from _version import __version__
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:

from _version import __version__
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()
    # delete incomplete experiments
    dbHandler.delete_incomplete_experiments()
print("Current code version, ", __version__)

Current code version,  13


In [77]:
# get the completed experiments with the current code version ordered by date
df_experiments = df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]
df_experiments = df_experiments.sort_values(by="date",ascending=False)
df_experiments.head(2)

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,tau,d,erm_methods,completed


In [78]:
# extract and print the top experiment_id
experiment_id = df_experiments.iloc[0]["experiment_id"]
print(experiment_id)

IndexError: single positional indexer is out-of-bounds

In [79]:
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == experiment_id]
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
gd = erm[erm["chosen_minimizer"] == "gd"]
# set alpha, epsilon, tau, lam as index in gd and state_evolution
gd = gd.set_index(["alpha","epsilon","tau","lam"])
state_evolution = state_evolution.set_index(["alpha","epsilon","tau","lam"])
# from state_evolution keep generalization_error, sigma, q, m, cosb
state_evolution = state_evolution[["generalization_error","sigma","q","m","cosb"]]
# add a suffix _state_evolution to the columns
state_evolution.columns = [col+"_state_evolution" for col in state_evolution.columns]
# from gd keep generalization_error_erm	generalization_error_overlap Q	rho	m	cosb
gd = gd[["generalization_error_erm","generalization_error_overlap","Q","rho","m","cosb"]]
# add a suffix _erm to the columns
gd.columns = [col+"_erm" for col in gd.columns]
# join the two dataframes
df = state_evolution.join(gd)
# reorder the columns
df = df[['generalization_error_state_evolution', 'generalization_error_erm_erm', 'generalization_error_overlap_erm', 'sigma_state_evolution', 'q_state_evolution','Q_erm','m_state_evolution','m_erm','cosb_state_evolution','cosb_erm', 'rho_erm']]
# average all quantities over the index and obtain also the standard deviation
df = df.groupby(level=[0,1,2,3]).agg([np.mean,np.std])
# display all columns
pd.set_option('display.max_columns', None)
df

generalization_error_state_evolution       \
                                                      mean  std   
alpha epsilon tau lam                                             
0.2   0.0     0.0 1.0                             0.480172  0.0   
1.0   0.0     0.0 1.0                             0.411124  0.0   
2.0   0.0     0.0 1.0                             0.344817  0.0   
5.0   0.0     0.0 1.0                             0.229622  0.0   

                      generalization_error_erm_erm            \
                                              mean       std   
alpha epsilon tau lam                                          
0.2   0.0     0.0 1.0                     0.390180  0.006549   
1.0   0.0     0.0 1.0                     0.272420  0.006041   
2.0   0.0     0.0 1.0                     0.205030  0.005591   
5.0   0.0     0.0 1.0                     0.124365  0.003326   

                      generalization_error_overlap_erm            \
                                                  mean       std   
alpha epsilon tau lam                                              
0.2   0.0     0.0 1.0                         0.389783  0.006147   
1.0   0.0     0.0 1.0                         0.271369  0.005682   
2.0   0.0     0.0 1.0                         0.205214  0.005478   
5.0   0.0     0.0 1.0                         0.123761  0.003433   

                      sigma_state_evolution      q_state_evolution       \
                                       mean  std              mean  std   
alpha epsilon tau lam                                                     
0.2   0.0     0.0 1.0              0.962750  0.0          0.032955  0.0   
1.0   0.0     0.0 1.0              0.839001  0.0          0.178136  0.0   
2.0   0.0     0.0 1.0              0.725744  0.0          0.379528  0.0   
5.0   0.0     0.0 1.0              0.528706  0.0          1.021613  0.0   

                          Q_erm           m_state_evolution          m_erm  \
                           mean       std              mean  std      mean   
alpha epsilon tau lam                                                        
0.2   0.0     0.0 1.0  0.033861  0.004110          0.062119  0.0  0.062110   
1.0   0.0     0.0 1.0  0.177491  0.003038          0.277949  0.0  0.274487   
2.0   0.0     0.0 1.0  0.382616  0.006015          0.492479  0.0  0.493459   
5.0   0.0     0.0 1.0  1.022788  0.006993          0.934955  0.0  0.930664   

                                cosb_state_evolution       cosb_erm            \
                            std                 mean  std      mean       std   
alpha epsilon tau lam                                                           
0.2   0.0     0.0 1.0  0.005084             0.062253  0.0  0.339317  0.018162   
1.0   0.0     0.0 1.0  0.008795             0.275598  0.0  0.657974  0.013350   
2.0   0.0     0.0 1.0  0.014888             0.468439  0.0  0.799164  0.010465   
5.0   0.0     0.0 1.0  0.021009             0.750896  0.0  0.925309  0.004063   

                        rho_erm            
                           mean       std  
alpha epsilon tau lam                      
0.2   0.0     0.0 1.0  0.993894  0.041168  
1.0   0.0     0.0 1.0  0.981194  0.047581  
2.0   0.0     0.0 1.0  0.997234  0.052464  
5.0   0.0     0.0 1.0  0.989516  0.042318

In [ ]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)